In [19]:
import sys
import subprocess
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import json
import builtins

from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.size'] = 14
plt.rcParams['text.usetex'] = True

### Count tree length with Sympy (not needed with new data)

In [20]:
# load sympy
from sympy import parse_expr

def count_nodes(expr):
        '''
        Counts the nodes of a sympy expression.

        Parameters
        ----------
        expr : sympy
                 sympy expression as created by SymExpr class.
        '''
        return sum(count_nodes(arg) for arg in expr.args) + 1
        # count = 0
        # for arg in expr.args:
        #     count += self.count_nodes(arg)
        # return count + 1


def sympy_expression_length(expr_string):
    expr = parse_expr(expr_string)
    return count_nodes(expr)

In [21]:
def read_results(path):
    frames = []

    for d in Path(path).iterdir():
        for f in d.iterdir():
            try:
                # df = pd.read_csv(f)
                df = pd.read_csv(f)
                df['backend'] = str(d).split('/')[-1]
                frames.append(df)
            except Exception as e:
                print(e)
                pass

    return pd.concat(frames)

### There results are generated with a Python script which wraps the Operon CLI programs and parses their output into Pandas data frames

In [22]:
df1 = read_results('./data/full_stat_runs/without_nls/') # results without NLS
df2 = read_results('./data/full_stat_runs/with_nls/')    # results with NLS

df = pd.concat([df1, df2])
df.shape


(10320892, 20)

In [23]:
df

,problem,symbols,generation,r2_train,r2_test,mae_train,mae_test,nmse_train,nmse_test,avg_fit,avg_len,eval,res_eval,jac_eval,opt_time,seed,elapsed,energy,expression,backend
0,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,0.0,0.387,0.085,4.0700,5.550,0.613000,0.915,2.580000e+38,42.3,2000.0,1000.0,0.0,0.0,3.157811e+09,0.001781,NaN,NaN,Mad_Transcendental_Faster
1,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,1.0,0.436,0.251,4.0800,5.100,0.564000,0.749,5.950000e+37,24.2,4000.0,2000.0,0.0,0.0,3.157811e+09,0.003214,NaN,NaN,Mad_Transcendental_Faster
2,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,2.0,0.514,0.337,3.7000,4.720,0.486000,0.663,-9.220000e-02,19.0,6000.0,3000.0,0.0,0.0,3.157811e+09,0.004559,NaN,NaN,Mad_Transcendental_Faster
3,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,3.0,0.580,0.159,3.4400,4.780,0.420000,0.841,-1.670000e-01,20.3,8000.0,4000.0,0.0,0.0,3.157811e+09,0.005901,NaN,NaN,Mad_Transcendental_Faster
4,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,4.0,0.595,0.354,3.4200,4.650,0.405000,0.646,-2.490000e-01,20.5,10000.0,5000.0,0.0,0.0,3.157811e+09,0.007218,NaN,NaN,Mad_Transcendental_Faster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218171,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,138.0,1.000,0.838,0.0145,0.402,0.000336,0.162,-9.160000e-01,39.6,278000.0,659970.0,317553.0,7187022.0,1.630222e+09,0.412368,NaN,NaN,Stl
218172,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,139.0,1.000,0.838,0.0145,0.402,0.000336,0.162,-9.150000e-01,39.6,280000.0,664742.0,319823.0,7248563.0,1.630222e+09,0.415653,NaN,NaN,Stl
218173,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,140.0,1.000,0.838,0.0145,0.402,0.000336,0.162,-9.140000e-01,39.2,282000.0,669515.0,322140.0,7310763.0,1.630222e+09,0.418974,NaN,NaN,Stl
218174,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,141.0,1.000,0.838,0.0145,0.402,0.000336,0.162,-9.150000e-01,39.3,284000.0,674296.0,324460.0,7371261.0,1.630222e+09,0.422244,NaN,NaN,Stl


In [24]:
len(df.expression.unique())

17908

### Check the actual $R^2$ (not within the fast framework but with Eigen/Stl)

Here, we call the `operon_parse_model` CLI tool, which will parse the model expressions and re-evaluate the output using the STL backend. The predictions are rescaled again using linear scaling.
This is a lengthy operation (the original data frame was 18000 rows), therefore we will parallelize it.

#### First, load problem data

This will need to be passed as arguments to `operon_parse_model`.

In [25]:
# we need to know some things about the data
data_path = './data/problems'

all_files  = list(os.listdir(data_path)) if os.path.isdir(data_path) else [ os.path.basename(data_path) ]
data_files = sorted([ f for f in all_files if f.endswith('.json') ])

problem_info = {}

# aggregate problem information that we can use to evaluate expressions
for f in data_files:
    with builtins.open(os.path.join(data_path, f), 'r') as h:
        info           = json.load(h)
        metadata       = info['metadata']
        target         = metadata['target']
        training_rows  = metadata['training_rows']
        training_start = training_rows['start']
        training_end   = training_rows['end']
        test_rows      = metadata['test_rows']
        test_start     = test_rows['start']
        test_end       = test_rows['end']
        problem_name   = metadata['name']
        problem_csv    = metadata['filename']

    problem_data = pd.read_csv(os.path.join(data_path, problem_csv))
    problem_info[problem_name] = {
        'name' : problem_name,
        'training_range' : (training_start, training_end),
        'test_range' : (test_start, test_end),
        'target' : target,
        'data' : problem_data
    }

problem_info

{'Airfoil Self-Noise': {'name': 'Airfoil Self-Noise',
  'training_range': (0, 1000),
  'test_range': (1000, 1503),
  'target': 'Y',
  'data':         X1    X2      X3    X4        X5        Y
  0      800   0.0  0.3048  71.3  0.002663  126.201
  1     1000   0.0  0.3048  71.3  0.002663  125.201
  2     1250   0.0  0.3048  71.3  0.002663  125.951
  3     1600   0.0  0.3048  71.3  0.002663  127.591
  4     2000   0.0  0.3048  71.3  0.002663  127.461
  ...    ...   ...     ...   ...       ...      ...
  1498  2500  15.6  0.1016  39.6  0.052849  110.264
  1499  3150  15.6  0.1016  39.6  0.052849  109.254
  1500  4000  15.6  0.1016  39.6  0.052849  106.604
  1501  5000  15.6  0.1016  39.6  0.052849  106.224
  1502  6300  15.6  0.1016  39.6  0.052849  104.204
  
  [1503 rows x 6 columns]},
 'Chemical-I': {'name': 'Chemical-I',
  'training_range': (0, 711),
  'test_range': (711, 1066),
  'target': 'y',
  'data':            x1       x2       x3       x4       x5       x6        x7  \
  0     5

In [26]:
from sklearn.metrics import r2_score
import hashlib
from scipy.stats import zscore

def problem_actual_name(problem_name):
    names = {
        'Airfoil Self-Noise' : 'AirfoilSelfNoise',
        'Concrete Compressive Strength' : 'Concrete',
        'Spatial Coevolution' : 'Pagie-1'
    }
    return names[problem_name] if problem_name in names else problem_name


def parse_float(s):
    try:
        f = float(s)
    except:
        f = np.nan
    return f


def evaluate_with_backend(backend, pname, expr, rg):
    a, b = rg
    cmd = [f'./bin/build_{backend}/cli/operon_parse_model', '--dataset', f'./data/problems/{pname}.csv', '--range', f'{a}:{b}', '--format', ':.20f', f"'{expr}'"]
    out = subprocess.check_output(cmd, shell=False)
    val = np.array([parse_float(x) for x in (v.decode('ascii') for v in out.split(b'\n')) if x != ''])
    return val


# useful in case we want to remember an expression by its hash to look at it later
def hash_expr(expr):
    try:
        h = hashlib.md5(expr.encode("utf-8")).hexdigest()[:7]
    except:
        h = 0

In [27]:
df.shape

(10320892, 20)

In [28]:
processed_rows = [None] * df.shape[0]

In [29]:
def fit_lstsq(y_pred, y_true):
    A = np.vstack([y_pred, np.ones(len(y_pred))]).T
    return np.linalg.lstsq(A, y_true, rcond=None)[0]


def evaluate_expr(problem, backend, expr):
    pass


def process_dataframe(df):
    for i, row in df.dropna(subset=['expression'], axis=0).iterrows():
        symbols = row['symbols']
        backend = row['backend']
        problem = row['problem']
        expr    = row['expression']
        expr_hash = hash_expr(expr)
        energy  = row['energy']

        # if np.isnan(energy):
        #     df.loc[i, 'r2_train_v'] = np.nan
        #     df.loc[i, 'r2_test_v'] = np.nan

        #     df.loc[i, 'r2_train_b'] = np.nan
        #     df.loc[i, 'r2_test_b'] = np.nan
        #     continue

        pname   = problem_actual_name(problem)
        pinfo   = problem_info[problem]
        data    = pinfo['data']

        training_start, training_end = pinfo['training_range']
        test_start, test_end = pinfo['test_range']

        # get the true target values from the dataset
        y_true_train = data.iloc[training_start:training_end,-1]
        y_true_test  = data.iloc[test_start:test_end,-1]

        y_stl_train = evaluate_with_backend('Stl', pname, expr, (training_start, training_end))
        y_stl_test  = evaluate_with_backend('Stl', pname, expr, (test_start, test_end))

        y_bak_train = evaluate_with_backend(backend, pname, expr, (training_start, training_end))
        y_bak_test  = evaluate_with_backend(backend, pname, expr, (test_start, test_end))

        np.nan_to_num(y_stl_train, copy=False)
        np.nan_to_num(y_stl_test,  copy=False)
        np.nan_to_num(y_bak_train, copy=False)
        np.nan_to_num(y_bak_test,  copy=False)

        # compute STL R2 values
        try:
            m, c = fit_lstsq(y_stl_train, y_true_train)
        except:
            m, c = 1, 0

        y_stl_train = y_stl_train * m + c
        y_stl_test = y_stl_test * m + c # reuse m, c from train

        try:
            r2_train_stl = r2_score(y_true_train, y_stl_train)
        except:
            r2_train_stl = 0

        try:
            r2_test_stl  = r2_score(y_true_test, y_stl_test)
        except:
            r2_test_stl  = 0

        # compute backend R2 values
        try:
            m, c = fit_lstsq(y_bak_train, y_true_train)
        except:
            m, c = 1, 0

        y_bak_train = y_bak_train * m + c
        y_bak_test = y_bak_test * m + c # reuse m, c from train

        try:
            r2_train_bak = r2_score(y_true_train, y_bak_train)
        except:
            r2_train_bak = 0

        try:
            r2_test_bak = r2_score(y_true_test, y_bak_test)
        except:
            r2_test_bak = 0

        df.loc[i, 'r2_train_v'] = r2_train_stl
        df.loc[i, 'r2_test_v'] = r2_test_stl

        df.loc[i, 'r2_train_b'] = r2_train_bak
        df.loc[i, 'r2_test_b'] = r2_test_bak

    return df


num_threads = 16
with Pool(num_threads) as p:
    parts = np.array_split(df, num_threads)
    results = p.map(process_dataframe, parts)

In [30]:
df = pd.concat(results)

In [31]:
df.to_csv('results_full.csv.gz', index=False, compression='gzip')

In [32]:
df.shape

(10320892, 24)

In [33]:
df1 = df[df.jac_eval == 0].dropna(subset=['expression'], axis=0).reset_index()
for k, v in df1.groupby(['problem', 'backend']):
    print(k, v['r2_test_v'].median())

('Airfoil Self-Noise', 'Eigen') 0.604055138990477
('Airfoil Self-Noise', 'Mad_Transcendental_Fast') 0.6300689140836542
('Airfoil Self-Noise', 'Mad_Transcendental_Faster') 0.586487242522215
('Airfoil Self-Noise', 'Mad_Transcendental_Fastest') 0.599438919471242
('Airfoil Self-Noise', 'Stl') 0.6179560104480541
('Airfoil Self-Noise', 'Vdt') 0.6470217701354558
('Chemical-I', 'Eigen') 0.6978559676775407
('Chemical-I', 'Mad_Transcendental_Fast') 0.7009939412575155
('Chemical-I', 'Mad_Transcendental_Faster') 0.7122741980622302
('Chemical-I', 'Mad_Transcendental_Fastest') 0.7272092144985605
('Chemical-I', 'Stl') 0.7083436533914131
('Chemical-I', 'Vdt') 0.7195703657637675
('Concrete Compressive Strength', 'Eigen') 0.48138655935268654
('Concrete Compressive Strength', 'Mad_Transcendental_Fast') 0.4777827880523948
('Concrete Compressive Strength', 'Mad_Transcendental_Faster') 0.4900229056540837
('Concrete Compressive Strength', 'Mad_Transcendental_Fastest') 0.47019971744395767
('Concrete Compressi

In [34]:
df2 = df[df.jac_eval > 0].dropna(subset=['expression'], axis=0).reset_index(drop=True)
df2.shape

(9000, 24)

In [35]:

df2.r2_train_v = df2.r2_train_v.fillna(0.0).clip(0, 1)
df2.r2_test_v = df2.r2_test_v.fillna(0.0).clip(0, 1)

for k, v in df2.groupby(['problem', 'backend']):
    print(k, v['r2_test_v'].median())

('Airfoil Self-Noise', 'Eigen') 0.5811912219303106
('Airfoil Self-Noise', 'Mad_Transcendental_Fast') 0.5658330196071115
('Airfoil Self-Noise', 'Mad_Transcendental_Faster') 0.5821834533765583
('Airfoil Self-Noise', 'Mad_Transcendental_Fastest') 0.5459158453538014
('Airfoil Self-Noise', 'Stl') 0.5718915301909114
('Airfoil Self-Noise', 'Vdt') 0.5706119177851965
('Chemical-I', 'Eigen') 0.811896665675653
('Chemical-I', 'Mad_Transcendental_Fast') 0.8147243841338243
('Chemical-I', 'Mad_Transcendental_Faster') 0.8151623594925014
('Chemical-I', 'Mad_Transcendental_Fastest') 0.8006965642089419
('Chemical-I', 'Stl') 0.8181609286291015
('Chemical-I', 'Vdt') 0.81305413775354
('Concrete Compressive Strength', 'Eigen') 0.4462766739707051
('Concrete Compressive Strength', 'Mad_Transcendental_Fast') 0.46242470401402663
('Concrete Compressive Strength', 'Mad_Transcendental_Faster') 0.45237935506016114
('Concrete Compressive Strength', 'Mad_Transcendental_Fastest') 0.4673385461012412
('Concrete Compressi

In [36]:
df2

,problem,symbols,generation,r2_train,r2_test,mae_train,mae_test,nmse_train,nmse_test,avg_fit,...,opt_time,seed,elapsed,energy,expression,backend,r2_train_v,r2_test_v,r2_train_b,r2_test_b
0,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,150.0,0.877,0.592,1.8200,3.430,0.123000,0.40800,-0.752,...,75865786.0,1.943992e+09,2.978762,213.00,((-0.063319958746433258056641) + (1.0005155801...,Mad_Transcendental_Faster,0.876565,0.592853,0.876770,0.592257
1,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,152.0,0.839,0.511,2.1200,3.730,0.161000,0.48900,-0.725,...,81079183.0,1.675822e+09,3.157546,221.00,((-0.001388675533235073089600) + (1.0000110864...,Mad_Transcendental_Faster,0.839028,0.510691,0.839029,0.510691
2,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,143.0,0.903,0.573,1.5900,3.370,0.096700,0.42700,-0.776,...,90548652.0,3.711168e+07,3.421617,247.71,((-0.081726528704166412353516) + (1.0006757974...,Mad_Transcendental_Faster,0.903090,0.575136,0.903291,0.572915
3,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,148.0,0.894,0.641,1.6800,3.180,0.106000,0.35900,-0.761,...,74271156.0,2.089412e+09,2.891453,209.22,((-2.560159921646118164062500) + (1.0204287767...,Mad_Transcendental_Faster,0.893804,0.642033,0.893843,0.641491
4,Airfoil Self-Noise,div_exp_log_sin_cos_sqrt_tanh,149.0,0.862,0.488,1.9200,3.680,0.138000,0.51200,-0.742,...,79392822.0,3.227517e+09,3.102702,218.57,((-0.078155986964702606201172) + (1.0006164312...,Mad_Transcendental_Faster,0.863514,0.493437,0.862469,0.487620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,139.0,0.999,0.827,0.0250,0.409,0.001210,0.17300,-0.889,...,6425966.0,3.646176e+08,0.383012,58.56,((-0.000121150114864576607943) + (1.0000343322...,Stl,0.998789,0.826594,0.998789,0.826594
8996,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,141.0,1.000,0.991,0.0133,0.116,0.000314,0.00909,-0.913,...,7667115.0,3.388603e+09,0.427565,66.44,(0.000002166297917938209139 + (1.0000050067901...,Stl,0.999686,0.990913,0.999686,0.990913
8997,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,141.0,1.000,0.833,0.0168,0.380,0.000478,0.16700,-0.909,...,6379898.0,3.318412e+09,0.379522,58.02,((-0.000867161026690155267715) + (1.0008423328...,Stl,0.999522,0.833338,0.999522,0.833338
8998,Vladislavleva-8,div_exp_log_sin_cos_sqrt_tanh,139.0,0.999,0.974,0.0182,0.166,0.000985,0.02630,-0.909,...,7036831.0,3.237736e+09,0.405360,63.04,(0.000374341994756832718849 + (1.0012280941009...,Stl,0.999015,0.973672,0.999015,0.973672
